In [1]:
import hermesql
from hermesql import terms
from hermesql.terms import *

In [2]:
from hermesql import Table
from hermesql.queries import *

# table

## class

In [20]:
class Table2(Selectable):
    @staticmethod
    def _init_schema(schema: Union[str, list, tuple, Schema, None]) -> Union[str, list, tuple, Schema, None]:
        # This is a bit complicated in order to support backwards compatibility. It should probably be cleaned up for
        # the next major release. Schema is accepted as a string, list/tuple, Schema instance, or None
        if isinstance(schema, Schema):
            return schema
        if isinstance(schema, (list, tuple)):
            return reduce(
                  lambda obj, s: Schema(s, parent=obj), schema[1:], Schema(schema[0])
            )
        if schema is not None:
            return Schema(schema)
        return None

    def __init__(self, table_name: str, alias: str, field: dict, criteria: Optional[dict] = None, schema: Optional[Union[Schema, str]] = None,  query_cls: Optional[Type["Query"]] = None) -> None:
        super().__init__(alias)
        self._table_name = table_name
        self._field = field
        self._criteria = criteria
        self._schema = self._init_schema(schema)
        self._query_cls = query_cls or Query
        if not issubclass(self._query_cls, Query):
            raise TypeError("Expected 'query_cls' to be subclass of Query")

            
    @classmethod
    def from_model(cls, model, table_name): return cls(*model.load_table(table_name))
            
    def get_table_name(self) -> str:
        return self.alias or self._table_name

    def get_sql(self, **kwargs: Any) -> str:
        quote_char = kwargs.get("quote_char")
        # FIXME escape
        table_sql = format_quotes(self._table_name, quote_char)

        if self._schema is not None:
            table_sql = "{schema}.{table}".format(
                  schema=self._schema.get_sql(**kwargs), table=table_sql
            )

        return format_alias_sql(table_sql, self.alias, **kwargs)

    def __str__(self) -> str:
        return self.get_sql(quote_char='"')

    def __eq__(self, other) -> bool:
        if not isinstance(other, Table):
            return False

        if self._table_name != other._table_name:
            return False

        if self._schema != other._schema:
            return False

        if self.alias != other.alias:
            return False

        return True

    def __repr__(self) -> str:
        if self._schema:
            return "Table('{}', schema='{}')".format(self._table_name, self._schema)
        return "Table('{}')".format(self._table_name)

    def __ne__(self, other: Any) -> bool:
        return not self.__eq__(other)

    def __hash__(self) -> int:
        return hash(str(self))

    def select(self, *terms: Sequence[Union[int, float, str, bool, Term, Field]]) -> "QueryBuilder":
        """
        Perform a SELECT operation on the current table

        :param terms:
            Type:  list[expression]

            A list of terms to select. These can be any type of int, float, str, bool or Term or a Field.

        :return:  QueryBuilder
        """
        return self._query_cls.from_(self).select(*terms)

    def update(self) -> "QueryBuilder":
        """
        Perform an UPDATE operation on the current table

        :return: QueryBuilder
        """
        return self._query_cls.update(self)

    def insert(self, *terms: Union[int, float, str, bool, Term, Field]) -> "QueryBuilder":
        """
        Perform an INSERT operation on the current table

        :param terms:
            Type: list[expression]

            A list of terms to select. These can be any type of int, float, str, bool or  any other valid data

        :return: QueryBuilder
        """
        return self._query_cls.into(self).insert(*terms)

## test create table

In [21]:
booking = Table2("booking", "b", {})

In [25]:
booking

Table('booking')

# Field_piece

## class

In [4]:
class Field_piece(Criterion, JSON):
    def __init__(self, name: str, calculation: Optional[str] = None, type: Optional[str] = None, table_alias: Optional[str] = None) -> None:
        self.alias = name
        self.calculation = calculation
        self.type = type
        self.table_alias = table_alias

    def nodes_(self) -> Iterator[NodeT]:
        yield self
        if self.table is not None:
            yield from self.table.nodes_()

    @builder
    def replace_table(self, current_table: Optional["Table"], new_table: Optional["Table"]) -> "Field":
        """
        Replaces all occurrences of the specified table with the new table. Useful when reusing fields across queries.

        :param current_table:
            The table to be replaced.
        :param new_table:
            The table to replace with.
        :return:
            A copy of the field with the tables replaced.
        """
        self.table = new_table if self.table == current_table else self.table

    def get_sql(self, with_alias = True, with_namespace = False, quote_char = None, **kwargs: Any) -> str:
        field_sql = format_quotes(self.calculation, quote_char)

        # Need to add namespace if the table has an alias
        if with_namespace or self.table_alias:
            field_sql = "{namespace}.{name}".format(
                  namespace=format_quotes(self.table_alias, quote_char), name=field_sql,
            )

        field_alias = getattr(self, "alias", None)
        if with_alias: return format_alias_sql(field_sql, field_alias, quote_char=quote_char, **kwargs)
        return field_sql

## test field piece

In [11]:
field_dict = {
    "invite_id": {
        "calculation": "id",
        "type": "dimension"
    },
    "invite_at": {
        "calculation": "fct.date_add(created_at)",
        "type": "dimension"
    },
    "invite_at_date": {
        "calculation": "fct.date_date_add(created_at)",
        "type": "dimension"
    }
}

In [67]:
field_dict_1 = {}

In [68]:
field_dict_1["invite_id"] = test_field

In [69]:
field_dict_1["invite_at"] = test_field

In [75]:
field_dict_1["invite_id"].get_sql()

'b.id invite_id'

In [71]:
booking.field = field_dict_1

In [65]:
test_field = Field_piece(name = "invite_id", calculation = "id", type = "dimension", table = booking)

In [66]:
test_field.get_sql()

'b.id invite_id'

# model

In [76]:
import json

## model dict

In [5]:
model_dict = {
    "table": {
        "invites": {
            "alias": "ivt",
            "field": {
                "invite_id": {
                    "calculation": "id",
                    "type": "dimension"
                },
                "invite_at": {
                    "calculation": "fct.date_add(created_at)",
                    "type": "dimension"
                },
                "invite_at_date": {
                    "calculation": "fct.date_date_add(created_at)",
                    "type": "dimension"
                },
                "referrer_id": {
                    "type": "dimension"
                },
                "referee_id": {
                    "calculation": "friend_id",
                    "type": "dimension"
                }
            },
            "criteria": ""
        },
        "invite_credit": {
            "name": "invite_credit",
            "alias": "ivt_crt",
            "field": {
                "dimension": {
                    "created_at": {
                        "calculation": "fct.date_add(created_at)",
                        "alias": "created_at"
                    },
                    "created_at_date": {
                        "calculation": "fct.date_date_add(created_at",
                        "alias": "created_at_date"
                    },
                    "invite_id": {
                        "calculation": "invite_id",
                        "alias": "invite_id"
                    },
                    "credit": {
                        "calculation": "credit",
                        "alias": "credit"
                    },
                    "status": {
                        "calculation": "status",
                        "alias": "status"
                    },
                    "type": {
                        "calculation": "type",
                        "alias": "type"
                    },
                    "booking_id": {
                        "calculation": "tybooking_idpe",
                        "alias": "booking_id"
                    },
                    "desc": {
                        "calculation": "`desc`",
                        "alias": "`desc`"
                    }
                },
                "measure": "",
                "criteria": {
                    "default": "invite_id is not null"                    
                }
            }
        }
    },

    "join": {

    },

    "function": {
        "date_add": "sql.date_add($field, interval + 7 hour)",
        "date_date_add": "sql.date(fct.date_add)"
    }
}

## class

In [6]:
class model:
    def __init__(self, model: dict):
        self.tables = model['table']
        self.joins = model['join']
        self.functions = model['function']        
        
    @classmethod
    def from_json_file(cls, file_path):
        f = open(file_path)
        json_file = f
        f.close()
        return cls.from_json(json_file)
        
    @classmethod
    def from_json(cls, json):
        model_dict = json.load(json) 
        return cls(model_dict)
        
    def load_table(self, table_name):
        self.table_alias = self.tables[table_name]['alias']
        fields = self.load_field_from_dict(self.tables[table_name]['field'])
        creteria = self.tables[table_name]['criteria']        
        return (table_name, self.table_alias, fields, creteria)
    
    def load_field_from_dict(self, field_dict):
        fields = {}
        for field in field_dict.keys():
            calculation = field_dict[field]['calculation'] if "calculation" in field_dict[field].keys() else None
            type = field_dict[field]['type']
            fields[field] = self.load_field(field, calculation, type, self.table_alias)         
        return fields
    
    def load_field(self, field_name, calculation, type, table_alias):
        return Field_piece(field_name, calculation, type, table_alias)
    
    def load_join(self, join_name): return None
    
    def load_function(sefl, function_name): return None

## test model

In [7]:
etl_model = model(model_dict)

In [11]:
invites = Table2.from_model(etl_model,"invites")

In [16]:
invites._field['invite_id'].calculation

'id'